In [6]:
# 'prebuilt-invoice', 'prebuilt-receipt', 'prebuilt-tax.us.w2', 'prebuilt-idDocument', 'prebuilt-businessCard'
model = 'prebuilt-invoice'

# path to pdf/image
container = 'landing'
folderpath = 'Invoices/'
filename = 'invoice.pdf'

StatementMeta(defaultSpark33, 0, 6, Finished, Available)

In [ ]:
pip install azure-ai-formrecognizer

In [3]:
import json
import pandas as pd
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

StatementMeta(defaultSpark33, 0, 3, Finished, Available)

In [4]:
# storage name
storageLinkedService = 'LS_LandingStorage'
storageAccount_ls = mssparkutils.credentials.getPropertiesAll(storageLinkedService)
storageAccountName = json.loads(storageAccount_ls)['Endpoint'].split('.')[0].replace('https://','')

#key vault name
keyVault_ls = mssparkutils.credentials.getPropertiesAll('LS_KeyVault')
keyVaultName = json.loads(keyVault_ls)['Endpoint'].split('.')[0].replace('https://','')

#cognitive services name and secret
cognitiveService_ls = mssparkutils.credentials.getPropertiesAll('LS_CognitiveService')
cognitiveServiceName = json.loads(cognitiveService_ls)['CsName']
cognitiveServiceKey = json.loads(cognitiveService_ls)['CsKey']

StatementMeta(defaultSpark33, 0, 4, Finished, Available)

In [8]:
mssparkutils.fs.mount( 
    f"abfss://{container}@{storageAccountName}.dfs.core.windows.net/{folderpath}", 
    "/mnt", 
    {"linkedService":f"{storageLinkedService}"} 
)

StatementMeta(defaultSpark33, 0, 8, Finished, Available)

True

In [9]:
jobid = mssparkutils.env.getJobId()

StatementMeta(defaultSpark33, 0, 9, Finished, Available)

In [10]:
with open(f"/synfs/{jobid}/mnt/{filename}", "rb") as fd:
    document = fd.read()

StatementMeta(defaultSpark33, 0, 10, Finished, Available)

In [11]:
document_analysis_client = DocumentAnalysisClient(
    endpoint=f"https://{cognitiveServiceName}.cognitiveservices.azure.com/",
    credential=AzureKeyCredential(cognitiveServiceKey)
)

StatementMeta(defaultSpark33, 0, 11, Finished, Available)

In [12]:
poller = document_analysis_client.begin_analyze_document(model, document)
result = poller.result()
# convert data to dictionary for processing
analyze_result_dict = result.to_dict()

StatementMeta(defaultSpark33, 0, 12, Finished, Available)

In [14]:
attribute_data = []
item_data = []

fields_output = {}

documents = analyze_result_dict.get('documents')

for document in documents:
    fields = document.get('fields').items()
    for name, field in fields:
        if name == "Items":
            # for each item (or row)
            for idx, item in enumerate(field.get('value')):
                row = {}
                # for each field in item (or row)
                for item_field_name, item_field in item.get('value').items():
                    if 'bounding_regions' in item_field.keys():
                        del item_field['bounding_regions']
                    if 'spans' in item_field.keys():
                        del item_field['spans']
                    value = json.dumps(item_field, default=str)
                    row[item_field_name] = value
                item_data.append(row)
        else:
            if 'bounding_regions' in field.keys():
                del field['bounding_regions']
            if 'spans' in field.keys():
                del field['spans']
            value = json.dumps(field, default=str)
            fields_output[name] = value

attribute_data.append(fields_output)

StatementMeta(defaultSpark33, 0, 14, Finished, Available)

In [15]:
item_df = pd.DataFrame(item_data)

StatementMeta(defaultSpark33, 0, 15, Finished, Available)

In [16]:
field_df = pd.DataFrame(attribute_data)

StatementMeta(defaultSpark33, 0, 16, Finished, Available)

In [17]:
if item_data == []:
    output = field_df
else:
    output = field_df.merge(item_df, how='cross')

StatementMeta(defaultSpark33, 0, 17, Finished, Available)

In [21]:
fileNoExt = filename.split('.')[0]
outputPath = f"abfss://{container}@{storageAccountName}.dfs.core.windows.net/{folderpath}{fileNoExt}_formrecognizeroutput.csv"
output.to_csv(outputPath, index=False, storage_options = {'linked_service' : f'{storageLinkedService}'})

StatementMeta(defaultSpark33, 0, 21, Finished, Available)